In [153]:
# Emoji Prediction using Transfer Learning & LSTM

In [154]:
import emoji
import numpy as np
import pandas as pd

In [155]:
#emoji.EMOJI_ALIAS_UNICODE

In [156]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":grinning_face_with_smiling_eyes:",
                    "3": ":disappointed_face:",
                    "4": ":fork_and_knife:",
                    "5": ":hundred_points:",
                    "6": ":fire:",
                    "7": ":face_blowing_a_kiss:",
                    "8": ":chestnut:",
                    "9": ":flexed_biceps:",
                   }

In [157]:
emoji.emojize(":fork_and_knife:")

'🍴'

In [158]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))


❤️
⚾
😄
😞
🍴
💯
🔥
😘
🌰
💪


In [159]:
train = pd.read_csv('dataset/train_emoji.csv',header=None)
test = pd.read_csv('dataset/test_emoji.csv',header=None)
data = train.values
print(data.shape)

(132, 4)


In [160]:
train.head()

0  1   2     3
0           never talk to me again  3 NaN   NaN
1  I am proud of your achievements  2 NaN   NaN
2   It is the worst day in my life  3 NaN   NaN
3                 Miss you so much  0 NaN   [0]
4                     food is life  4 NaN   NaN

In [161]:
X_train = train[0]
Y_train = train[1]
X_test = test[0]
Y_test = test[1]
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😄
The assignment is too long  😞
I want to go play ⚾


In [162]:
# Converting sentences into Embeddings using Glove 6B.50d.txt

In [163]:
f = open('glove.6B.50d.txt',encoding='utf-8')

In [164]:
embeddings_index = {}
for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings_index[word] = coeffs
f.close()        
        

In [165]:
embeddings_index["eat"]
embeddings_index["eat"].shape

(50,)

In [166]:
emb_dim = embeddings_index["eat"].shape[0]

In [167]:
# Converting sentences into vectors (Embedding Layer Output)

In [168]:
def embedding_output(X):
    maxLen = 10
    embedding_out = np.zeros((X.shape[0],maxLen,emb_dim))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            # every word in the current(ix) sentence
            embedding_out[ix][jx] = embeddings_index[X[ix][jx].lower()]
            
    return embedding_out

In [169]:
embedding_matrix_train = embedding_output(X_train)
embedding_matrix_test = embedding_output(X_test)

<ipython-input-168-7d81e7756281>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix] = X[ix].split()


In [170]:
print(X_train[0])
print(len(X_train[0]))
print(embedding_matrix_train.shape)
print(embedding_matrix_test.shape)

['never', 'talk', 'to', 'me', 'again']
5
(132, 10, 50)
(56, 10, 50)


In [171]:
from keras.utils import to_categorical

In [172]:
Y_train = to_categorical(Y_train,num_classes=5)
Y_test = to_categorical(Y_test,num_classes=5)
print(Y_train.shape)
#print(Y_test[0])

(132, 5)


In [173]:
# Define the RNN/LSTM Model

In [174]:
from keras.layers import *
from keras.models import Sequential

In [180]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,input_shape=(10,50),return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 10, 64)            29440     
_________________________________________________________________
dropout_16 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_17 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 325       
_________________________________________________________________
activation_11 (Activation)   (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________

In [181]:
from keras.callbacks import EarlyStopping 
from keras.callbacks import ModelCheckpoint
#checkpoint = ModelCheckpoint("best_model.h5",monitor = "val_loss",verbose= True,save_best_only = True)
#earlystop= EarlyStopping(monitor = "val_acc",patience=10)
hist = model.fit(embedding_matrix_train,Y_train,batch_size=64,epochs=150,shuffle=True,validation_split=0.2)

Epoch 1/150
2/2 [==============================] - 2s 931ms/step - loss: 1.6178 - acc: 0.2381 - val_loss: 1.6034 - val_acc: 0.1852
Epoch 2/150
2/2 [==============================] - 0s 25ms/step - loss: 1.5732 - acc: 0.3048 - val_loss: 1.6038 - val_acc: 0.1852
Epoch 3/150
2/2 [==============================] - 0s 21ms/step - loss: 1.5226 - acc: 0.3429 - val_loss: 1.6161 - val_acc: 0.1481
Epoch 4/150
2/2 [==============================] - 0s 22ms/step - loss: 1.4919 - acc: 0.3810 - val_loss: 1.6373 - val_acc: 0.1852
Epoch 5/150
2/2 [==============================] - 0s 21ms/step - loss: 1.4803 - acc: 0.4000 - val_loss: 1.6546 - val_acc: 0.1481
Epoch 6/150
2/2 [==============================] - 0s 21ms/step - loss: 1.4327 - acc: 0.3524 - val_loss: 1.6719 - val_acc: 0.1481
Epoch 7/150
2/2 [==============================] - 0s 23ms/step - loss: 1.4091 - acc: 0.4381 - val_loss: 1.6780 - val_acc: 0.1481
Epoch 8/150
2/2 [==============================] - 0s 21ms/step - loss: 1.4015 - acc: 0.4

2/2 [==============================] - 0s 22ms/step - loss: 0.0342 - acc: 1.0000 - val_loss: 1.5556 - val_acc: 0.6296
Epoch 64/150
2/2 [==============================] - 0s 20ms/step - loss: 0.0319 - acc: 1.0000 - val_loss: 1.5208 - val_acc: 0.6667
Epoch 65/150
2/2 [==============================] - 0s 19ms/step - loss: 0.0446 - acc: 0.9810 - val_loss: 1.3920 - val_acc: 0.7037
Epoch 66/150
2/2 [==============================] - 0s 19ms/step - loss: 0.0237 - acc: 1.0000 - val_loss: 1.2435 - val_acc: 0.7407
Epoch 67/150
2/2 [==============================] - 0s 24ms/step - loss: 0.0278 - acc: 0.9905 - val_loss: 1.1823 - val_acc: 0.7407
Epoch 68/150
2/2 [==============================] - ETA: 0s - loss: 0.0245 - acc: 0.984 - 0s 23ms/step - loss: 0.0296 - acc: 0.9905 - val_loss: 1.2164 - val_acc: 0.7407
Epoch 69/150
2/2 [==============================] - 0s 22ms/step - loss: 0.0195 - acc: 1.0000 - val_loss: 1.2942 - val_acc: 0.7407
Epoch 70/150
2/2 [==============================] - 0s 23m

Epoch 125/150
2/2 [==============================] - 0s 23ms/step - loss: 0.0050 - acc: 1.0000 - val_loss: 1.7648 - val_acc: 0.6296
Epoch 126/150
2/2 [==============================] - 0s 21ms/step - loss: 0.0060 - acc: 1.0000 - val_loss: 1.9392 - val_acc: 0.5926
Epoch 127/150
2/2 [==============================] - 0s 22ms/step - loss: 0.0051 - acc: 1.0000 - val_loss: 2.0582 - val_acc: 0.5926
Epoch 128/150
2/2 [==============================] - 0s 23ms/step - loss: 0.0140 - acc: 0.9905 - val_loss: 2.0380 - val_acc: 0.5926
Epoch 129/150
2/2 [==============================] - 0s 22ms/step - loss: 0.0045 - acc: 1.0000 - val_loss: 2.0049 - val_acc: 0.6296
Epoch 130/150
2/2 [==============================] - 0s 24ms/step - loss: 0.0047 - acc: 1.0000 - val_loss: 1.9606 - val_acc: 0.6667
Epoch 131/150
2/2 [==============================] - 0s 23ms/step - loss: 0.0064 - acc: 1.0000 - val_loss: 1.8813 - val_acc: 0.7037
Epoch 132/150
2/2 [==============================] - 0s 22ms/step - loss: 0.

In [182]:
pred = model.predict_classes(embedding_matrix_test)

In [183]:
model.evaluate(embedding_matrix_test,Y_test)

2/2 [==============================] - 0s 3ms/step - loss: 2.3398 - acc: 0.5893


[2.3397772312164307, 0.5892857313156128]

In [184]:
for i in range(30):
    print(' '.join(X_test[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a raise
😄
😞
she got me a present
❤️
😄
ha ha ha it was so funny
😄
😄
he is a good friend
❤️
😄
I am upset
❤️
⚾
We had such a lovely dinner tonight
❤️
😄
where is the food
🍴
🍴
Stop making this joke ha ha ha
😄
😄
where is the ball
⚾
⚾
work is hard
😞
😄
This girl is messing with me
😞
❤️
are you serious ha ha
😄
❤️
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😄
Congratulation for having a baby
😄
😄
stop messing around
😞
😞
any suggestions for dinner
🍴
😄
I love taking breaks
❤️
❤️
you brighten my day
😄
❤️
I boiled rice
🍴
🍴
she is a bully
😞
❤️
Why are you feeling bad
😞
😞
I am upset
😞
⚾
I worked during my birthday
😞
😄
My grandmother is the love of my life
❤️
❤️
enjoy your break
😄
🍴
valentine day is near
❤️
😄
